In [55]:
import numpy as np
import pandas as pd

In [2]:
np.random.seed(2)

dataset_size = ["small", "medium", "large"][0]

dataset_info = {
    "small": {
        "dataset_name": "wine",
        "class_name": "Class",
        "drop_fields": []
    },
    "medium": {
        "dataset_name": "breast-cancer-wisconsin",
        "class_name": "Class",
        "drop_fields": ["Sample code number"]
    },
    "large": {
        "dataset_name": "seismic-bumps",
        "class_name": "class",
        "drop_fields": []
    }
}

dataset_name = dataset_info[dataset_size]["dataset_name"]
class_name = dataset_info[dataset_size]["class_name"]
drop_fields = dataset_info[dataset_size]["drop_fields"]

df = pd.read_csv('../data/' + dataset_name + ".csv")
df = df.drop(drop_fields, axis=1)
df = df.iloc[np.random.permutation(len(df))]

n_cut = int(0.8*len(df))
df_trn = df[:n_cut]
df_tst = df[n_cut:]

X_trn = df_trn.drop(class_name, axis=1)
y_trn = df_trn[class_name]

X_tst = df_tst.drop(class_name, axis=1)
y_tst = df_tst[class_name]

In [2]:
from CART import CART

X = [['blue', 'blonde', 'tall'],
     ['blue', 'brown', 'medium'],
     ['brown', 'brown', 'medium'],
     ['green', 'brown', 'medium'],
     ['green', 'brown', 'tall'],
     ['brown', 'brown', 'low'], 
     ['green', 'blonde', 'low'],
     ['blue', 'brown', 'medium']]

X = np.array(X)
y = np.array([1, 1, 0, 0, 1, 0, 0, 1])


clf = CART(max_depth=10)
clf.fit(X, y)
clf.predict(X)

(array(['brown'], dtype='<U6'), array(['blonde'], dtype='<U6')) 0.5
(array(['low'], dtype='<U6'), array(['medium', 'tall'], dtype='<U6')) 0.3333333333333333
(array(['low', 'tall'], dtype='<U6'), array(['medium'], dtype='<U6')) 0.5
(array(['low', 'medium'], dtype='<U6'), array(['tall'], dtype='<U6')) 0.33333333333333337
(array(['blue'], dtype='<U6'), array(['brown', 'green'], dtype='<U6')) 0.1999999999999999
(array(['blue', 'green'], dtype='<U6'), array(['brown'], dtype='<U6')) 0.3333333333333333
(array(['blue', 'brown'], dtype='<U6'), array(['green'], dtype='<U6')) 0.4666666666666667
(array(['brown'], dtype='<U6'), array(['blonde'], dtype='<U6')) 0.30000000000000004
(array(['green'], dtype='<U6'), array(['brown'], dtype='<U6')) 0.26666666666666666
(array(['low'], dtype='<U6'), array(['medium', 'tall'], dtype='<U6')) 0.26666666666666666
(array(['low', 'tall'], dtype='<U6'), array(['medium'], dtype='<U6')) 0.26666666666666666
(array(['low', 'medium'], dtype='<U6'), array(['tall'], dtype=

array([1, 1, 0, 0, 1, 0, 0, 1])

In [194]:
y

array([1, 1, 0, 0, 1, 0, 0, 1])